# Importing and preparing supermarkets data

## Libraries and settings

In [1]:
# Libraries
import os
import fnmatch
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Get current working directory
print('Current working directory:', os.getcwd())

# Show .json files in the current working directory
flist = fnmatch.filter(os.listdir('.'), '*.json')
for i in flist:
    print(i)

Current working directory: /workspaces/data_analytics/Week_02
supermarkets.json


## Importing data

In [2]:
# Read the data to a pandas data frame
df1 = pd.read_json('supermarkets.json', encoding='utf-8')
df1.head(5)

,type,id,lat,lon,tags
0,node,33126515,47.155616,9.037915,"{'brand': 'Spar', 'brand:wikidata': 'Q610492',..."
1,node,36726161,47.226191,8.980329,"{'addr:city': 'Uznach', 'addr:housenumber': '2..."
2,node,39768209,47.225069,8.969981,"{'addr:city': 'Uznach', 'addr:postcode': '8730..."
3,node,39947904,47.376732,8.542161,"{'addr:city': 'Zürich', 'addr:country': 'CH', ..."
4,node,48932835,47.375020,8.522895,"{'addr:city': 'Zürich', 'addr:housenumber': '7..."


## Count number of rows and columns in the data frame

In [3]:
# Dimension (rows, columns)
print('Dimension:', df1.shape)

# Number of rows
print('Number of rows:', df1.shape[0])

# Number of columns
print('Number of columns:', df1.shape[1])

Dimension: (3392, 5)
Number of rows: 3392
Number of columns: 5


## Column 'tags' is a pandas Series with dictionaries -> change to data frame

In [4]:
# Type of the first item of column tags
print(type(df1.tags))
print(type(df1.tags[0]))

# Content of the first item of column tags
print(df1.tags[0].keys())

# Change to data frame
df2 = pd.DataFrame.from_records(df1.tags)
df2 = df2[['brand', 'shop', 'addr:city', 'addr:street', 'addr:housenumber', 'addr:postcode']]

# Rename selected columns
df2 = df2.rename(columns={'addr:city': 'city',
                          'addr:street':'street',
                          'addr:housenumber': 'housenumber',
                          'addr:postcode': 'postcode'})

# Show first records of data frame
df2.head()

<class 'pandas.core.series.Series'>
<class 'dict'>
dict_keys(['brand', 'brand:wikidata', 'brand:wikipedia', 'name', 'opening_hours', 'shop'])


,brand,shop,city,street,housenumber,postcode
0,Spar,supermarket,NaN,NaN,NaN,NaN
1,Migros,supermarket,Uznach,Zürcherstrasse,25,8730
2,Coop,supermarket,Uznach,NaN,NaN,8730
3,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
4,Migros,supermarket,Zürich,Wengistrasse,7,8004


## Merge df1 and df2

In [5]:
# Merge df and df2
df = pd.merge(df1[['type', 'id', 'lat', 'lon']], 
              df2[['brand', 'shop', 'city', 'street', 'housenumber', 'postcode']],
              left_index=True, 
              right_index=True)
df.head(5)

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730
2,node,39768209,47.225069,8.969981,Coop,supermarket,Uznach,NaN,NaN,8730
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004


## Count and identify the number of missing values (if any)

In [6]:
# Count missing values
print(pd.isna(df).sum())

# Identify rows with missing values, e.g.:
df.loc[pd.isna(df['city'])]

type              0
id                0
lat               0
lon               0
brand          1065
shop              0
city           1777
street         1608
housenumber    1680
postcode       1709
dtype: int64


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN
5,node,60271452,47.406671,9.305450,NaN,supermarket,NaN,NaN,NaN,NaN
6,node,70656485,47.491253,8.733981,NaN,supermarket,NaN,NaN,NaN,NaN
10,node,81321513,47.532917,9.066408,Landi,supermarket,NaN,NaN,NaN,NaN
13,node,95582038,47.050385,9.059214,NaN,supermarket,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3384,node,11083317088,46.862184,9.531169,Lidl,supermarket,NaN,NaN,NaN,NaN
3386,node,11098091830,46.205111,6.130174,Coop,supermarket,NaN,NaN,NaN,NaN
3387,node,11099817248,46.928691,7.561873,NaN,supermarket,NaN,NaN,NaN,NaN
3388,node,11103235832,46.166742,8.771970,Migros,supermarket,NaN,NaN,NaN,NaN


## Count and identify duplicated values (if any)

In [7]:
# Count duplicated values
print(df.duplicated().sum())

# Identify rows with duplicated values, e.g.:
df[df[['id']].duplicated()]

0


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode


## Get data types of all variables

In [8]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

type            object
id               int64
lat            float64
lon            float64
brand           object
shop            object
city            object
street          object
housenumber     object
postcode        object
dtype: object

## Additional filters on supermarkets

### Filter only Migros supermarkets in the city of Zürich

In [9]:
# Filter Migros supermarkets in Zürich
df_migros_zurich = df.loc[(df['brand'] == 'Migros') & (df['city'] == 'Zürich')]
print(f'Number of Migros supermarkets in Zürich: {df_migros_zurich.shape[0]}')
df_migros_zurich.head()

Number of Migros supermarkets in Zürich: 30


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004
11,node,83330862,47.344749,8.529981,Migros,supermarket,Zürich,Etzelstrasse,3,8038
16,node,119249170,47.375255,8.536107,Migros,supermarket,Zürich,Löwenstrasse,31-35,8001
50,node,262400822,47.364072,8.530945,Migros,supermarket,Zürich,Tessinerplatz,10,8002
71,node,267346993,47.385598,8.531471,Migros,supermarket,Zürich,Limmatstrasse,152,8005


### Filter and count all Coop supermarkets in the cities of Zürich, Basel & Bern

In [10]:
# Filter Coop supermarkets in Zürich, Basel & Bern
df_coop_cities = df.loc[(df['brand'] == 'Coop') & (df['city'].isin(['Zürich', 'Basel', 'Bern']))]
print(f'Number of Coop supermarkets in Zürich, Basel & Bern: {df_coop_cities.shape[0]}')
print('\nBreakdown by city:')
print(df_coop_cities['city'].value_counts())
df_coop_cities.head()

Number of Coop supermarkets in Zürich, Basel & Bern: 52

Breakdown by city:
city
Zürich    37
Basel     10
Bern       5
Name: count, dtype: int64


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
9,node,79977755,47.340070,8.530546,Coop,supermarket,Zürich,Alte Kalchbühlstrasse,15,8038
59,node,265776668,47.376417,8.559594,Coop,supermarket,Zürich,Zürichbergstrasse,75,8044
63,node,266630559,47.377716,8.511219,Coop,supermarket,Zürich,Badenerstrasse,333,8003
70,node,267345511,47.385809,8.516574,Coop,supermarket,Zürich,Maagplatz,1,8005


### Filter supermarkets with available brand, city, house number and postcode

In [11]:
# Filter supermarkets with complete address information
df_complete = df.loc[df['brand'].notna() & 
                     df['city'].notna() & 
                     df['housenumber'].notna() & 
                     df['postcode'].notna()]
print(f'Number of supermarkets with complete address information: {df_complete.shape[0]}')
print(f'Percentage of complete records: {df_complete.shape[0] / df.shape[0] * 100:.2f}%')
df_complete.head()

Number of supermarkets with complete address information: 1148
Percentage of complete records: 33.84%


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004
7,node,70656488,47.491874,8.706448,Migros,supermarket,Winterthur,Zürcherstrasse,102,8406
8,node,75749133,47.340967,8.530601,ALDI,supermarket,Zürich,Albisstrasse,81,8038


### Include opening hours as additional variable in the data frame

In [12]:
# Extract opening_hours from the original df1.tags column
df1_tags = pd.DataFrame.from_records(df1.tags)

# Check if opening_hours column exists
if 'opening_hours' in df1_tags.columns:
    # Add opening_hours to the main dataframe
    df['opening_hours'] = df1_tags['opening_hours']
    print(f'Opening hours added to dataframe')
    print(f'Number of supermarkets with opening hours: {df["opening_hours"].notna().sum()}')
    print(f'Number of supermarkets without opening hours: {df["opening_hours"].isna().sum()}')
else:
    print('Opening hours information not available in the data')

# Show sample of opening hours
print('\nSample of opening hours:')
print(df[df['opening_hours'].notna()][['brand', 'city', 'opening_hours']].head(10))

Opening hours added to dataframe
Number of supermarkets with opening hours: 2031
Number of supermarkets without opening hours: 1361

Sample of opening hours:
     brand        city                                      opening_hours
0     Spar         NaN  Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00
1   Migros      Uznach  Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17...
3     Coop      Zürich                                  Mo-Sa 06:00-22:00
4   Migros      Zürich                          Mo-Sa 08:00-21:00; PH off
7   Migros  Winterthur          Mo-Fr 07:30-20:00; PH off; Sa 08:00-19:00
8     ALDI      Zürich                                  Mo-Sa 07:30-21:00
9     Coop      Zürich                                  Mo-Sa 07:30-21:00
10   Landi         NaN                     Mo-Sa 08:00-12:00, 13:30-18:00
11  Migros      Zürich                          Mo-Sa 07:30-21:00; PH off
13     NaN         NaN                           Mo-Su 09:00-21:00;PH off


### Filter supermarkets with available opening hours

In [13]:
# Filter supermarkets with available opening hours
df_with_hours = df.loc[df['opening_hours'].notna()]
print(f'Number of supermarkets with opening hours: {df_with_hours.shape[0]}')
print(f'Percentage of supermarkets with opening hours: {df_with_hours.shape[0] / df.shape[0] * 100:.2f}%')

# Show distribution by brand
print('\nDistribution by brand:')
print(df_with_hours['brand'].value_counts().head(10))

# Display sample data
df_with_hours.head()

Number of supermarkets with opening hours: 2031
Percentage of supermarkets with opening hours: 59.88%

Distribution by brand:
brand
Migros                449
Coop                  358
Denner                250
Volg                  166
ALDI                   60
Lidl                   50
Spar                   47
VOI Migros Partner     33
Aldi                   27
VOI                    20
Name: count, dtype: int64


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode,opening_hours
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17..."
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001,Mo-Sa 06:00-22:00
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004,Mo-Sa 08:00-21:00; PH off
7,node,70656488,47.491874,8.706448,Migros,supermarket,Winterthur,Zürcherstrasse,102,8406,Mo-Fr 07:30-20:00; PH off; Sa 08:00-19:00


## Save data to file

In [14]:
df.to_csv('supermarkets_data_prepared.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [15]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1030-azure
Datetime: 2025-12-07 07:37:37
Python Version: 3.11.14
-----------------------------------
